In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Conectar ao MT5
if not mt5.initialize():
    print("Falha na inicialização do MetaTrader5")
    mt5.shutdown()
    quit()

# Configurar o símbolo e o timeframe
symbol = "PETR4"
timeframe = mt5.TIMEFRAME_M1

In [ ]:
# Definir os parâmetros do algoritmo
ema_period_short = 12
ema_period_long = 26
bb_period = 20
bb_deviation = 2
rsi_period = 10

In [ ]:
# Método para calcular a MME curta
def calculate_ema_short(data):
    ema_short = pd.Series(index=data.index)
    ema_short.iloc[0] = data.iloc[0]
    for i in range(1, len(data)):
        ema_short.iloc[i] = (2 / (ema_period_short + 1)) * data.iloc[i] + (1 - 2 / (ema_period_short + 1)) * ema_short.iloc[i-1]
    return ema_short

# Método para calcular a MME longa
def calculate_ema_long(data):
    ema_long = pd.Series(index=data.index)
    ema_long.iloc[0] = data.iloc[0]
    for i in range(1, len(data)):
        ema_long.iloc[i] = (2 / (ema_period_long + 1)) * data.iloc[i] + (1 - 2 / (ema_period_long + 1)) * ema_long.iloc[i-1]
    return ema_long

# Método para calcular as Bandas de Bollinger
def calculate_bollinger_bands(data):
    sma = data.rolling(bb_period).mean()
    std = data.rolling(bb_period).std()
    upper_band = sma + bb_deviation * std
    lower_band = sma - bb_deviation * std
    return upper_band, sma, lower_band

# Método para calcular o RSI
def calculate_rsi(data):
    delta = data.diff()
    gain = delta.copy()
    gain[gain < 0] = 0
    loss = -delta.copy()
    loss[loss < 0] = 0
    avg_gain = gain.rolling(rsi_period).mean()
    avg_loss = loss.rolling(rsi_period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
import pytz
from datetime import datetime

# Obter os dados do símbolo e timeframe
# rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 200)

timezone = pytz.timezone("America/Sao_Paulo")
# utc_from = datetime(2023, 5, 22, hour = 10, minute = 24, tzinfo=timezone)
utc_from = datetime(2023, 5, 23, tzinfo=timezone)
utc_to = datetime(2023, 5, 24, tzinfo=timezone)
# utc_to = datetime(2023, 5, 25, hour = 10, tzinfo=timezone)

rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)

In [ ]:
# Criar um dataframe com os dados
df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)

In [ ]:
def ema(data):
    ema2 = 50
    ema_long = pd.Series(index=data.index)
    ema_long.iloc[0] = data.iloc[0]
    for i in range(1, len(data)):
        ema_long.iloc[i] = (2 / (ema2 + 1)) * data.iloc[i] + (1 - 2 / (ema2 + 1)) * ema_long.iloc[i-1]
    return ema_long

In [ ]:
# Calcular os indicadores
ema_short = calculate_ema_short(df['close'])
ema_long = calculate_ema_long(df['close'])
upper_band, sma, lower_band = calculate_bollinger_bands(df['close'])
rsi = calculate_rsi(df['close'])
# ema = ema(df['close'])

df['ema_short'] = ema_short
df['ema_long'] = ema_long
df['upper_band'] = upper_band
df['lower_band'] = lower_band
df['sma'] = sma
df['rsi'] = rsi

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
# Imprimir os resultados no console
print("Preço: \n", df['close'])
print("EMA Short: \n", ema_short)
print("EMA Long: \n", ema_long)
print("SMA: \n", sma)
print("Upper Band: \n", upper_band)
print("Lower Band: \n", lower_band)
print("RSI: \n", rsi)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(df['close'])

In [ ]:
#Plotar os resultados
plt.figure(figsize=(25, 7))
plt.plot(ema_short, color='blue')
plt.plot(ema_long, color='black')
plt.plot(upper_band, color='green')
plt.plot(df['close'], color='purple')
plt.plot(lower_band, color='red')
# plt.plot(ema, color='red')
plt.fill_between(df.index, lower_band, upper_band, alpha=0.1)
plt.title('RETORNO ACUMULADO')
plt.xlabel('Data/Hora')
plt.ylabel('RETORNO ACUMULADO')
plt.legend(['MME Curta', 'MME Longa', 'Banda Superior', 'Preço', 'Banda Inferior'])
plt.show()

In [ ]:
plt.figure(figsize=(25, 3))
plt.title('RSI')
plt.xlabel('Data/Hora')
plt.plot(rsi, color='orange')

In [ ]:
df[['close','lower_band', 'upper_band', 'ema_short', 'ema_long']].plot(figsize=(20, 6))

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
df.iplot(kind='candle')

In [ ]:
qf = cf.QuantFig(df, title='Setup', legend='top', name='Candlestick')
qf.add_bollinger_bands(periods=20,boll_std=2)
qf.add_rsi(periods=20,color='java')
qf.add_macd()
qf.iplot()

In [ ]:
#Do vídeo: df = Criando Candlestick com Python Matplotlib
import mplfinance as mpf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
print(mpf.available_styles())

In [ ]:
df_sem_indicadores = pd.DataFrame(rates)
df_sem_indicadores['time'] = pd.to_datetime(df_sem_indicadores['time'], unit='s')
df_sem_indicadores.set_index('time', inplace=True)

In [ ]:
df_sem_indicadores

In [ ]:
mpf.plot(df_sem_indicadores.tail(100), type='candle', figsize=(16, 8), style='yahoo')

In [ ]:
df

In [ ]:
df[['close','lower_band', 'upper_band', 'ema_short', 'ema_long']].plot(figsize=(20, 6))

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(rsi, color='orange')

In [ ]:
#Calcular os sinais de compra e venda EMA
def ema_signal():
    df["signal"] = 0
# df.loc[(df["ema_short"] > df["ema_long"]), "signal"] = 1
# df.loc[(df["ema_short"] < df["ema_long"]), "signal"] = -1

    df["signal"] = np.where(df["ema_short"] > df["ema_long"], 1, np.where(df["ema_short"] < df["ema_long"], -1, 0))

    crossings = df["signal"].diff()
    df["signal"] = np.where(crossings != 0, crossings.cumsum(), 0)

In [ ]:
# Calcular os sinais de compra e venda DE EMA + BB + RSI
df["signal"] = 0
df.loc[(df["ema_short"] > df["ema_long"]) & (df["close"] < df["lower_band"]) & (df["rsi"] < 20), "signal"] = 1
df.loc[(df["ema_short"] < df["ema_long"]) & (df["close"] > df["upper_band"]) & (df["rsi"] > 80), "signal"] = -1

In [ ]:
df

In [ ]:
df["signal"].plot(figsize=(20, 5))

In [ ]:
df

In [ ]:
# Calcular os sinais de compra e venda BB + RSI
df["signal"] = 0
df.loc[(df["close"] < df["lower_band"]) & (df["rsi"] < 20), "signal"] = 1
df.loc[(df["close"] > df["upper_band"]) & (df["rsi"] > 80), "signal"] = -1

In [ ]:
df

In [ ]:
df["signal"].plot(figsize=(20, 5))

In [ ]:
# Calcular os sinais de compra e venda EMA + RSI
df["signal"] = 0
df.loc[(df["ema_short"] > df["ema_long"]) & (df["rsi"] < 20), "signal"] = 1
df.loc[(df["ema_short"] < df["ema_long"]) & (df["rsi"] > 80), "signal"] = -1

In [ ]:
df["signal"].plot(figsize=(20, 5))

In [ ]:
#Calcular os sinais de compra e venda EMA
df["signal"] = 0
# df.loc[(df["ema_short"] > df["ema_long"]), "signal"] = 1
# df.loc[(df["ema_short"] < df["ema_long"]), "signal"] = -1

df["signal"] = np.where(df["ema_short"] > df["ema_long"], 1, np.where(df["ema_short"] < df["ema_long"], -1, 0))

crossings = df["signal"].diff()
df["signal"] = np.where(crossings != 0, crossings.cumsum(), 0)

In [ ]:
df

In [ ]:
df["signal"].plot(figsize=(20, 5))

In [ ]:
df

In [ ]:
df[['close','ema_short', 'ema_long']].plot(figsize=(20, 6))

In [ ]:
#Calcular os sinais de compra e venda BB
df["signal"] = 0
df.loc[(df["close"] < df["lower_band"]), "signal"] = 1
df.loc[(df["close"] > df["upper_band"]), "signal"] = -1

In [ ]:
df["signal"].plot(figsize=(20, 5))

In [ ]:
df

In [ ]:
#Calcular os sinais de compra e venda RSI
df["signal"] = 0
df.loc[(df["rsi"] < 20), "signal"] = 1
df.loc[(df["rsi"] > 80), "signal"] = -1

In [ ]:
df["signal"].plot(figsize=(20, 5))

In [ ]:
df

In [ ]:
# import time
#
# lot_size = 1000
# initial_balance = 10000
# # lot = 0.10
# # result = 0
# # order = None
# # price = None
# request = {}
#
# # df['ema_short'] = df['close'].ewm(span=ema_period_short, adjust=False).mean()
# # df['ema_long'] = df['close'].ewm(span=ema_period_long, adjust=False).mean()
# # df['bb_upper'], df['bb_middle'], df['bb_lower'] = \
# #     ta.bollinger_bands(df['close'], bb_period, bb_deviation)
# # df['rsi'] = ta.rsi(df['close'], rsi_period)
#
# # Definição da estratégia de investimento
# df['signal'] = 0
# df.loc[(df['ema_short'] > df['ema_long']) &
#          (df['close'] < df['lower_band']) &
#          (df['rsi'] < 30), 'signal'] = 1
# df.loc[(df['ema_short'] < df['ema_long']) &
#          (df['close'] > df['upper_band']) &
#          (df['rsi'] > 70), 'signal'] = -1
#
# # Criação de um novo DataFrame com informações sobre as operações de compra/venda
# positions = pd.DataFrame(index=df.index)
# positions['signal'] = df['signal']
# positions['close'] = df['close']
# positions['rsi'] = df['rsi']
# positions['bb_upper'] = df['upper_band']
# positions['bb_middle'] = df['sma']
# positions['bb_lower'] = df['lower_band']
# positions['ema_short'] = df['ema_short']
# positions['ema_long'] = df['ema_long']
# positions['position'] = positions['signal'].diff()
# positions['position'].iloc[0] = positions['signal'].iloc[0]
# positions = positions[positions['position'] != 0]
#
# # Realização do backtesting
# balance = initial_balance
# trades = []
# for i, row in positions.iterrows():
#     print("aqui")
#     if row['position'] == 1:
#         order = mt5.ORDER_TYPE_BUY
#         price = mt5.symbol_info_tick(symbol).ask
#         print(f"Compra realizada em {i} por {price}")
#     elif row['position'] == -1:
#         order = mt5.ORDER_TYPE_SELL
#         price = mt5.symbol_info_tick(symbol).bid
#         print(f"Venda realizada em {i} por {price}")
#
#     lot = lot_size
#     request = {
#         "action": mt5.TRADE_ACTION_DEAL,
#         "symbol": symbol,
#         "volume": lot,
#         "type": order,
#         "price": price,
#         "deviation": 100,
#         "magic": 123456,
#         "comment": "Python script open",
#         "type_time": mt5.ORDER_TIME_GTC,
#         "type_filling": mt5.ORDER_FILLING_IOC,
#     }
#
# # Envio da ordem de compra ou venda
# result = mt5.order_send(request)
#
# if result.retcode != mt5.TRADE_RETCODE_DONE:
#     print(f"Erro ao enviar ordem: {result.comment}")
#
# # Se a ordem foi enviada com sucesso, aguarda sua execução
# while True:
#     # Configurar o símbolo e o timeframe
#     symbol = "EURUSD"
#     timeframe = mt5.TIMEFRAME_M5
#
#     # Obter os dados do símbolo e timeframe
#     rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 100)
#
#     # Obtém informações sobre a ordem
#     order = mt5.orders_get(result.order)
#
#     print("ESPERANDO EXECUÇÃO DA ORDEM")
#
#     if order == mt5.TRADE_RETCODE_DONE:
#         break
# #     time.sleep(1)ff

In [ ]:
# import time
#
#
# symbol = "EURUSD"
# lot = 0.1
# magic_number = 123456
# comment = "Signal from Python"
#
# # get the current price
# price = mt5.symbol_info_tick(symbol).ask
#
# # open a position
# request = {
#     "action": mt5.TRADE_ACTION_DEAL,
#     "symbol": symbol,
#     "volume": lot,
#     "type": mt5.ORDER_TYPE_BUY,
#     "price": price,
#     "magic": magic_number,
#     "comment": comment,
#     "type_time": mt5.ORDER_TIME_GTC,
#     "type_filling": mt5.ORDER_FILLING_FOK,
# }
# result = mt5.order_send(request)
# if result.retcode != mt5.TRADE_RETCODE_DONE:
#     print("order_send failed, retcode={}".format(result.retcode))
#     mt5.shutdown()
#     quit()
#
# # wait for the order to be filled
# while True:
#     deals = mt5.history_deals_get(symbol=symbol)
#     if len(deals) > 0:
#         last_deal = deals[-1]
#         if last_deal.magic == magic_number and last_deal.entry == mt5.DEAL_ENTRY_IN:
#             print("Position opened: ticket={}".format(last_deal.order))
#             break
#     time.sleep(1)

In [ ]:
#Portfolio para backtesting
#Setup EMA
import vectorbt as vbt

buy_signal = (df["ema_short"] > df["ema_long"])
sell_signal = (df["ema_short"] < df["ema_long"])

pf = vbt.Portfolio.from_signals(df['close'], buy_signal, sell_signal)

In [ ]:
pf.plot().show()

In [ ]:
pf.stats()

In [ ]:
#Portifolio para trading
#Setup RSI

buy_signal = (df["rsi"] < 20)
sell_signal = (df["rsi"] > 80)

pf = vbt.Portfolio.from_signals(df['close'], buy_signal, sell_signal)

In [ ]:
pf.plot().show()

In [ ]:
pf.stats()

In [ ]:
buy_signal

In [ ]:
#Portifolio para trading
#Setup Bollinger Bands

buy_signal = (df["close"] < df["lower_band"])
sell_signal = (df["close"] > df["upper_band"])

pf = vbt.Portfolio.from_signals(df['close'], buy_signal, sell_signal)

In [ ]:
pf.plot().show()

In [ ]:
pf.stats()

In [ ]:
#Portifolio para trading
#Setup Bollinger Bands + RSI

buy_signal = (df["close"] < df["lower_band"]) & (df["rsi"] < 20)
sell_signal = (df["close"] > df["upper_band"]) & (df["rsi"] > 80)

pf = vbt.Portfolio.from_signals(df['close'], buy_signal, sell_signal)

In [ ]:
pf.plot().show()

In [ ]:
pf.stats()

In [ ]:
#Portifolio para trading
#Setup RSI + EMA

buy_signal = (df["rsi"] < 20) & (df["ema_short"] > df["ema_long"])
sell_signal = (df["rsi"] > 80) & (df["ema_short"] < df["ema_long"])

pf = vbt.Portfolio.from_signals(df['close'], buy_signal, sell_signal)

In [ ]:
pf.plot().show()

In [ ]:
pf.stats()

In [ ]:
#Portifolio para trading
#Setup Bollinger Bands + RSI + EMA

buy_signal = (df["close"] < df["lower_band"]) & (df["rsi"] < 20) & (df["ema_short"] > df["ema_long"])
sell_signal = (df["close"] > df["upper_band"]) & (df["rsi"] > 80) & (df["ema_short"] < df["ema_long"])

pf = vbt.Portfolio.from_signals(df['close'], buy_signal, sell_signal)

In [ ]:
pf.plot().show()

In [ ]:
pf.stats()